In [8]:
{
    "prompt":"Item=handbag, Color=army_green, price=$99, size=S->", 
    "completion":"This stylish small green handbag will add a unique touch to your look, without costing you a fortune."
}

{'prompt': 'Item=handbag, Color=army_green, price=$99, size=S->',
 'completion': 'This stylish small green handbag will add a unique touch to your look, without costing you a fortune.'}

In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L12-v2')

query_embedding = model.encode(input('What is your question?'))
passage_embedding = model.encode(['London has 9,787,426 inhabitants at the 2011 census',
                                  'London is known for its finacial district',
                                  'London is 500 square kilometers',
                                  'The quality of this batch is very bad'])


x = util.dot_score(query_embedding, passage_embedding)
print("Similarity:", x[[0]])

Similarity: tensor([[0.4588, 0.6045, 0.7027, 0.0325]])


In [10]:
print(x[0])

tensor([0.4588, 0.6045, 0.7027, 0.0325])


In [11]:
import torch

torch.softmax(x[0], dim=-1)

tensor([0.2447, 0.2831, 0.3123, 0.1598])

In [13]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)
print(res)
# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)



{'score': 0.2117147594690323, 'start': 59, 'end': 84, 'answer': 'gives freedom to the user'}
